In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('https://gist.githubusercontent.com/DiogoRibeiro7/c6590d0cf119e87c39e31c21a9c0f3a8/raw/4a8e3da267a0c1f0d650901d8295a5153bde8b21/PlayTennis.csv')

In [3]:
df.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [4]:
pd.crosstab(df['Outlook'],df['Play Tennis'],normalize='columns').stack().to_dict()

{('Overcast', 'No'): 0.0,
 ('Overcast', 'Yes'): 0.4444444444444444,
 ('Rain', 'No'): 0.4,
 ('Rain', 'Yes'): 0.3333333333333333,
 ('Sunny', 'No'): 0.6,
 ('Sunny', 'Yes'): 0.2222222222222222}

In [5]:
D={}
for i in ['Outlook','Temperature',	'Humidity',	'Wind']:
    D.update(pd.crosstab(df[i], df['Play Tennis'],normalize='columns').stack().to_dict())


In [6]:
D

{('Overcast', 'No'): 0.0,
 ('Overcast', 'Yes'): 0.4444444444444444,
 ('Rain', 'No'): 0.4,
 ('Rain', 'Yes'): 0.3333333333333333,
 ('Sunny', 'No'): 0.6,
 ('Sunny', 'Yes'): 0.2222222222222222,
 ('Cool', 'No'): 0.2,
 ('Cool', 'Yes'): 0.3333333333333333,
 ('Hot', 'No'): 0.4,
 ('Hot', 'Yes'): 0.2222222222222222,
 ('Mild', 'No'): 0.4,
 ('Mild', 'Yes'): 0.4444444444444444,
 ('High', 'No'): 0.8,
 ('High', 'Yes'): 0.3333333333333333,
 ('Normal', 'No'): 0.2,
 ('Normal', 'Yes'): 0.6666666666666666,
 ('Strong', 'No'): 0.6,
 ('Strong', 'Yes'): 0.3333333333333333,
 ('Weak', 'No'): 0.4,
 ('Weak', 'Yes'): 0.6666666666666666}

# sentimental-analysis using naive bayes

In [8]:
import numpy as np
import pandas as pd

In [9]:
df=pd.read_csv("/media/kibria/624309e6-cae1-4eca-a603-4ebc61fc0d9b/data/IMDB Dataset.csv")

In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa


Text Cleaning¶

    Sample 10000 rows
    Remove html tags
    Remove special characters
    Converting every thing to lower case
    Removing Stop words
    Stemming



In [12]:
df=df.sample(10000)

In [14]:
df.shape

(10000, 2)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 49104 to 15749
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [16]:
df['sentiment'].replace({"positive":1,"negative":0},inplace=True)

/tmp/ipykernel_33603/2922232226.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'].replace({"positive":1,"negative":0},inplace=True)


In [17]:
df.head()

,review,sentiment
49104,This is a very odd film ... I wasn't really su...,0
30194,The price of a dream - and some dreams can be ...,0
30740,"First, an explanation: Despite my headline, I'...",1
15633,"As romantic comedies go, this was a cute and w...",1
25933,This God forsaken film is about three dumb mil...,0


In [18]:
import re
clean=re.compile('<.*?>')
re.sub(clean,'',df.iloc[2].review)

'First, an explanation: Despite my headline, I\'m giving this film only 8 stars because overall this is NOT one of the best films ever made. All the criticisms registered here have valid points. Also, be warned that to enjoy the script you really need to appreciate Neil Simon\'s brilliance with finding the wit within real human banter. He does have a distinctively New York ear for dialogue -- especially dry, Jewish, love-suffused sarcasm -- and if you have trouble accepting sarcasm as an expression of love, then you might have trouble accepting the optimism at the heart of this movie.So much for warnings. Here\'s my main point: Walter Matthau is flat-out perfect, even beyond perfect, in this movie. I have never seen him funnier, or more touching for that matter -- because at the same time that he shows us the hilariousness of this character who refuses to give up his Big Star self-image or insufferable attitudes even as his coherence is in decline, he also shows us the more vulnerable,

In [19]:
# Function to clean html tags
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [20]:
df['review']=df['review'].apply(clean_html)

In [21]:
def convert_lower(text):
    return text.lower()

In [22]:
df['review']=df['review'].apply(convert_lower)

In [23]:

def remove_special(text):
    x=''
    
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x + ' '
    return x

In [24]:
df['review']=df['review'].apply(remove_special)

In [25]:
import nltk

In [26]:
from nltk.corpus import stopwords

In [27]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [29]:
def remove_stopwords(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:] 
    x.clear()
    return y       

In [30]:
df['review']=df['review'].apply(remove_stopwords)

In [32]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [34]:
y=[]
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z    

In [35]:
stem_words(['I','loved','loving','it'])

['i', 'love', 'love', 'it']

In [36]:
df['review']=df['review'].apply(stem_words)


In [37]:
def join_back(list_input):
    return " ".join(list_input)

In [38]:
df.head()

,review,sentiment
49104,"[odd, film, realli, sure, n, london, lowlif, f...",0
30194,"[price, dream, dream, expens, view, english, t...",0
30740,"[first, explan, despit, headlin, give, film, 8...",1
15633,"[romant, comedi, go, cute, win, one, thought, ...",1
25933,"[god, forsaken, film, three, dumb, millionair,...",0


In [39]:
df['review']=df['review'].apply(join_back)

In [40]:
df.head()

,review,sentiment
49104,odd film realli sure n london lowlif find mute...,0
30194,price dream dream expens view english translat...,0
30740,first explan despit headlin give film 8 star o...,1
15633,romant comedi go cute win one thought write co...,1
25933,god forsaken film three dumb millionair gather...,0


In [41]:
X=df.iloc[:,0:1].values

In [42]:
X.shape

(10000, 1)

In [44]:
X

array([['odd film realli sure n london lowlif find mute kid wood believ kind oracl somehow make way chang someth live usual end disast film end 90 minut leav feel quit unsatisfi almost annoy pointless care charact none get chanc endear viewer messag dim odd happen runner emilio shoot kid made absolut sens pointless someon help understand mess film'],
       ['price dream dream expens view english translat version perhap reason low rate viewer made overal film poorer stori materi hint comment seem suggest subtitl version would better plot element remain unexplain leav unfinish feel also leav thought seri follow piti stage least anyway'],
       ['first explan despit headlin give film 8 star overal one best film ever made critic regist valid point also warn enjoy script realli need appreci neil simon brillianc find wit within real human banter distinct new york ear dialogu especi dri jewish love suffus sarcasm troubl accept sarcasm express love might troubl accept optim heart movi much w

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500)

In [46]:
X=cv.fit_transform(df['review']).toarray()

In [47]:
X.shape

(10000, 2500)

In [48]:
y=df.iloc[:,-1].values

In [49]:
y.shape

(10000,)

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [51]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [52]:
clf1=GaussianNB()
clf2=MultinomialNB()
clf3=BernoulliNB()


In [53]:
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)

BernoulliNB()

In [54]:
y_pred1=clf1.predict(X_test)
y_pred2=clf2.predict(X_test)
y_pred3=clf3.predict(X_test)


In [55]:
from sklearn.metrics import accuracy_score

In [56]:
print("Gaussian",accuracy_score(y_test,y_pred1))
print("Multinomial",accuracy_score(y_test,y_pred2))
print("Bernaulli",accuracy_score(y_test,y_pred3))

Gaussian 0.7195
Multinomial 0.845
Bernaulli 0.842
